In [ ]:
%%capture
%config Completer.use_jedi = False
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

# Install on Google Colab
import subprocess
import sys

from IPython import get_ipython

install_packages = "google.colab" in str(get_ipython())
if install_packages:
    for package in ["qrules[doc]", "graphviz"]:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", package]
        )

# Calculation of LS couplings

We want to take a look how calculate the possible LS combinations for a certain decay.
For this we use the decay $J/\psi\to\bar\Sigma(1580)^-\Sigma^+$ where the $\bar\Sigma(1580)^-$ decays into $\bar p$ and $K^0$.
Both these decays are strong interactions.

## $J/\psi\to\bar\Sigma(1580)^-\Sigma^+$

The first transition $J/\psi\to\bar\Sigma(1580)^-\Sigma^+$ can be expressed in quantum numbers as $1^-\to\frac{3}{2}^+\frac{1}{2}^+$.
By coupling the spins of the decay products we can determine the spin of the transition.

$$
\begin{eqnarray}
|s_{\bar\Sigma(1580)^-} - s_{\Sigma^+}| & \le S & \le |s_{\bar\Sigma(1580)^-} + s_{\Sigma^+}|\\
|\frac{3}{2}-\frac{1}{2}| & \le S & \le |\frac{3}{2} + \frac{1}{2}|\\
1 & \le S & \le 2
\end{eqnarray}
$$

Since $\Delta S=1$ we have 2 possible solutions for $S$, 1 and 2.

Next step is to couple angular momentum and the spin to the total momentum.
$$
\begin{eqnarray}
|L-S| & \le J & \le |L+S| \\
|L-1| & \le 1 & \le |L+1| \\
      & \text{or} \\
|L-2| & \le 1 & \le|L+2|
\end{eqnarray}
$$

$L$ can be an integer multiple of 1, e.g. 1,2,3....
Now we have to check which of this $L$'s solves this conditions.
For $S=1$ $L$ can be 1 or 2.
For $S=2$ the results are 1,2 and 3.
So in total we have 6 LS combinations.
$$(L,S)=(0,1),(1,1),(2,1),(1,2),(2,2),(3,2)$$

But this decays by the strong force.
This means parity has to be conserved and not all of this transitions are possible.
\begin{align*}
\eta_{J/\psi}=&\eta_{\bar\Sigma(1580)^-}\cdot\eta_{\Sigma^+}\cdot(-1)^L\\
(-1)=&(+1)\cdot(-1)\cdot(-1)^L\\
(+1)=&(-1)^{L}
\end{align*}
From this we can easily see that only odd $L$'s are possible.
$$(L,S)=(1,1),(1,2),(3,2)$$

## $\bar \Sigma(1580)^-\to \bar pK^0$

But this only the first part of the decay.
The $\bar\Sigma(1580)^-$ decays further into $\bar p$ and $K^0$.
In quantum numbers this can be expressed as $\frac{3}{2}^+\to\frac{1}{2}^-0^-$.

Again let's start with by coupling the spins of the child particles.
$$
\begin{eqnarray}
|s_{\bar p} - s_{K^0}| & \le S  & \le |s_{\bar p} + s_{K^0}|\\
|\frac{1}{2}-0| & \le S & \le |\frac{1}{2} + 0|\\
S=\frac{1}{2}
\end{eqnarray}
$$
This time only one spin is possible.

Next is to couple the angular momentum and the spin again.

$$
\begin{eqnarray}
|L-S| & \le J & \le|L+S|\\
|L-\frac{1}{2}| & \le \frac{3}{2} & \le|L+\frac{1}{2}|
\end{eqnarray}
$$

Like before the way is to check which integer solves this condition.
Here 2 $LS$ combinations are possible:

$$
(L,S)=(1,\frac{1}{2}),(2,\frac{1}{2})
$$

This again is a strong interaction which means we have to check for parity conservation.

$$
\begin{align*}
    \eta_{\bar \Sigma(1580)^-}=&\eta_{\bar p}\cdot\eta_{K^0}\cdot(-1)^L\\
    (+1)=&(-1)\cdot(-1)\cdot(-1)^L\\
    (+1)=&(-1)^L
\end{align*}
$$

Here it is clear only even $L$'s are allowed.
Which means one $LS$ combination is possible.

$$
(L,S)=(2,\frac{1}{2})
$$